In [118]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [113]:
df1 = pd.read_csv('./Rdry_RH_wp_Rwet.csv')
df2 = pd.read_csv('./Refractive_Index_from_ARIA/Fractive_index_LUT.csv')
df3 = pd.read_csv('./wavelength.csv')

In [114]:
# tempture is set as 240 k

temp = 240 # K

### add wavelength to RH_wp_Rdry_Rwet

### To find the N, iK: 

First, compare the wp 

Second, comare wavelength

FYI: df2 is sorted from:

    wp: from low to high
    
        temp: from low to high
        
            wavelength: from low to high

In [115]:
def find_N_iK(Array5):
    # [wavelength_nm, RH_%, wp_%, Rdry_um, Rwet_um]
#     print(Array5.shape)
    
    wavelength = Array5[0]
    RH   = Array5[1]
    wp   = Array5[2]
    Rdry = Array5[3]
    Rwet = Array5[4]
    
    # find closest temp in df2
    idx = (np.abs(df2_WPs - wp)).argmin()
    wp2 = df2_WPs[idx]
#     print(wp2)
    
    # for all df2 data with wp2, select closest temp
    df2_tmp1 = df2.loc[df2["wt_%"] == wp2]
    df2_Ts = df2_tmp1["temp_K"].unique()
    idx = (np.abs(df2_Ts - temp)).argmin()
    T2 = df2_Ts[idx] 
#     print(T2)
    
    # for all df2 data with wp2 and T2, select closest wavelength
    df2_tmp2 = df2_tmp1.loc[ df2["temp_K"] == T2 ]
    df2_wv = df2_tmp2["wavl_nm"].unique()
    idx = (np.abs(df2_wv - wavelength)).argmin()
    wavelength2 = df2_wv[idx]
#     print(wavelength2)
    
    # select the df2 row with wp2, T2 and wavelength2
    df2_tmp3 = df2_tmp2.loc[ df2["wavl_nm"] == wavelength2 ]
    [[N, iK]] = df2_tmp3[["n","k"]].values
#     print([N, iK])
    
    return [N, iK]

In [116]:
N_Rdry = 40
N_RH = 7
N_wv = len(wavelength)

if N_Rdry*N_RH!=len(df1):
    print("ERROR: dimension doesn't match !!!")

data = np.zeros((N_Rdry*N_RH*N_wv, 7)) # wavelength_nm, RH_%, wp_%, Rdry_um, Rwet_um

In [119]:
# data: [wavelength_nm, RH_%, wp_%, Rdry_um, Rwet_um]

for i in tqdm(range(N_Rdry)):
    for j in range(N_wv):
        for k in range(N_RH):
            
            df1_index = i*N_RH + k
            
            index = i*N_wv*N_RH + j*N_RH + k
            
            data[index,0] = wavelength[j]
#             print(k,j,df1_index)
            data[index,1:5] = df1.iloc[df1_index][1:5].values
    
            data[index,5:7] = find_N_iK(data[index,0:5])
                
                

    

100%|██████████| 40/40 [05:38<00:00,  8.47s/it]


In [120]:
df = pd.DataFrame(data)
df.columns =["w(nm)", "RH", "wp_%", "Rdry_um", "Rwet_um", "N", "iK"]

df.to_csv('h2so4_total.csv')

In [121]:
df

,w(nm),RH,wp_%,Rdry_um,Rwet_um,N,iK
0,300.0,0.000394,0.0,99.000000,0.000394,1.33259,0.00000
1,300.0,0.000394,50.0,39.842581,0.000591,1.37965,0.01137
2,300.0,0.000394,70.0,30.549615,0.000659,1.41790,0.01732
3,300.0,0.000394,80.0,25.469146,0.000709,1.42001,0.01367
4,300.0,0.000394,90.0,17.952168,0.000811,1.42001,0.01367
...,...,...,...,...,...,...,...
80355,7082.2,3.225682,70.0,30.549615,5.393621,1.25531,0.18306
80356,7082.2,3.225682,80.0,25.469146,5.798730,1.22907,0.13133
80357,7082.2,3.225682,90.0,17.952168,6.636533,1.22907,0.13133
80358,7082.2,3.225682,95.0,15.000000,7.099407,1.22907,0.13133


In [48]:
df1.iloc[0][1:5].values

array([3.9422487e-04, 0.0000000e+00, 9.9000000e+01, 3.9422487e-04])

In [38]:
wavelength = df3["wavelength"]
print(len(wavelength))

print(len(df1))

287
280


In [122]:
df1.describe()

,Unnamed: 0,Rdry_um,RH_%,wp_%,Rwet_um
count,280.000000,280.000000,280.000000,280.000000,280.000000
mean,139.500000,0.390860,69.142857,34.687644,0.693933
std,80.973247,0.743080,32.264270,27.617561,1.361762
min,0.000000,0.000394,0.000000,15.000000,0.000394
25%,69.750000,0.003764,50.000000,15.000000,0.006212
50%,139.500000,0.035877,80.000000,25.469146,0.061730
75%,209.250000,0.340824,95.000000,39.842581,0.613271
max,279.000000,3.225682,99.000000,99.000000,7.099407


In [27]:
df2.describe()

,Unnamed: 0,n,k,wt_%,temp_K,wavn_cm-1,wavl_micro-m,wavl_nm
count,171970.000000,171970.000000,171970.000000,171970.000000,171970.000000,171970.000000,171970.000000,171970.000000
mean,2153.488358,1.390006,0.106799,57.507542,248.082224,3889.849705,4.060842,4060.841976
std,1402.194928,0.112368,0.149312,18.552271,31.561029,1995.096151,3.787847,3787.846950
min,0.000000,1.048270,-0.003470,10.000000,203.000000,413.492750,1.333596,1333.596349
25%,892.000000,1.344220,0.008440,38.000000,223.000000,2173.123900,1.796985,1796.985005
50%,2036.000000,1.379140,0.016870,65.000000,243.000000,3868.250250,2.585148,2585.148156
75%,3380.000000,1.410410,0.159710,72.000000,273.000000,5564.876710,4.601670,4601.670434
max,4723.000000,2.072740,0.922540,81.000000,298.000000,7498.520830,24.184221,24184.220884


In [28]:
df3.describe()

,wavelength
count,287.000000
mean,5596.980488
std,9514.122517
min,229.800000
25%,675.000000
50%,2044.200000
75%,6968.600000
max,55555.600000


In [29]:
Rdry = df1["Rdry_um"]

In [33]:
# Rdry[0:40]

In [34]:
# Rdry[40:80]